# An Interactive Dashboard to understand GDP, Population and Life Expectancy of the World

#### Author : Thashma Nellira Ragunanda

I am using the plotly dash library of python to create a dash app of GDP,population and life expectancy of different countries.Written on top of Plotly.js and React.js, Dash is ideal for building and deploying data apps with customized user interfaces. I have also used plotly express and the geography related functions to create a couple of contextual visualizations.

I also used pandas to create dataframes using the csv files,clean and convert specif colums to their data types and filter out the necessary information for the plots.

I am using 2 different csv datasets for my visualizations which can be found at

https://github.com/thashmadech/IS445/blob/main/project/gapminder.csv

https://github.com/thashmadech/IS445/blob/main/project/countries_codes_and_coordinates.csv


##### 1. Gapminder :
Gapminder identifies systematic misconceptions about important global trends and proportions and uses reliable data to develop easy to understand teaching materials to rid people of their misconceptions. I have downloaded the file and used 3 different indicators for my visualization.

I obtained it from Harvard dataverse. It is based on free material from GAPMINDER.ORG, CC-BY LICENSE.

The data obtained is for the years 1952 - 2007

##### 2. Countries and codes: 
A file downloaded from https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes which countains the Alpha-3 and Alpha-2 codes for countries



In [1]:
# essential imports
from jupyter_dash import JupyterDash 

import dash
from dash import html
from dash import dcc
from dash.dependencies import Output, Input

import plotly.express as px
import math
from dash import no_update

import pandas as pd
import numpy as np
import json

In [2]:
# read the GDP data
df_country = pd.read_csv("https://github.com/thashmadech/IS445/blob/main/project/gapminder.csv?raw=true")

In [3]:
df_country.head();

In [4]:
# we load a secondary dataset with all countries and their 3-letter alpha code 
df_country_code = pd.read_csv("https://github.com/thashmadech/IS445/blob/main/project/countries_codes_and_coordinates.csv?raw=true")
df_country_code['Alpha-3 code'] = df_country_code['Alpha-3 code'].apply(lambda s : s.replace('"', ""))

In [5]:
df_country_code.head();

In [6]:
#available economic indicators in the gapminder.csv file

indicators = ['lifeExp','pop','gdpPercap']

I am using the below helper function to display a single row from the Countries and codes dataframe as well as display the name of the country whose data has been hovered and clicked on the scatter plot.

In [7]:
# a helper function
def get_country_name(country_code):    
  one_row = df_country_code[df_country_code['Alpha-3 code'].str.strip() == country_code]
  if not one_row.empty:
    display(one_row)
    return one_row['Country'].values[0]
  else:
    return ''
# end

The below css is available at codepen.io which can be used to create rows and columns. I am using them to create a layout for my dashboard

In [8]:
# this external css creates columns and row layout
style_css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

###  Dashboard Design:

In Dash, multiple html elements are added inside an html div tag as a list. The list construct takes classnames to evenly space the columns. For example we can use classname = "three columns" of we want to span the space into 3 columns.

Below, I am creating a dashboard template with rows and columns that I need. I have created 3 rows.

1st row: The Header

2nd row: Two drop downs and 2 radio boxes

3rd row: A scatter plot, a slider and a bar plot


In [9]:
app = JupyterDash(__name__, external_stylesheets=style_css)

app.layout = html.Div([
    # first row: header
    html.H4('An interactive Dashboard of GDP Vs Population Vs life Expectancy '),

    # second row: two drop-downs and radio-boxes. 
    html.Div([
      html.Div([
        dcc.Dropdown(
          id='xaxis-column',
          options=[{'label': i, 'value': i} for i in indicators], #e.g., {label: 'pop', 'value':'pop'}
          value='lifeExp'
        ),
        dcc.RadioItems(
          id='xaxis-type',
          options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
          value='Linear',
          labelStyle={'display': 'inline-block'}
        )
      ], className='four columns'),

      html.Div([
        dcc.Dropdown(
          id='yaxis-column',
          options=[{'label': i, 'value': i} for i in indicators],
          value='gdpPercap'
        ),
        dcc.RadioItems(
          id='yaxis-type',
          options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
          value='Linear',
          labelStyle={'display': 'inline-block'}
        )
      ], className='four columns')

    ], className='row'),


    # third row:
    html.Div([

      # first item: scatter plot
      html.Div([

        # add scatter plot
        dcc.Graph(
          id='scatter-plot',
          figure=px.scatter() 
        ),

        # add slider
        dcc.Slider(
          id='year-slider',
          min=df_country['year'].min(),
          max=df_country['year'].max(),
          value=df_country['year'].min(),
          marks={str(year): str(year) for year in df_country['year'].unique()},
          step=None
        )

      ], className='seven columns'),

      

      # second item: bar chart
      html.Div([
        dcc.Graph(
          id='bar-chart',
          figure=px.bar()
        )
      ], className='five columns')

    ], className = 'row'),

    

    # fourth row
    html.Div([
        html.Div([
#           html.H3('Debug'),
          #html.Br(),
          html.P(id='output_text_1', children='Total:'),
          html.P(id='output_text_2', children='Details:')
#           html.P(id='output_text_3', children='Details2:'),
#           html.P(id='output_text_4', children='Details3:')
        ], className = 'five columns')

    ], className = 'row')

])




#### Callbacks

I am creating 2 call backs for the 2 plots on my dashboard.

Each callback  has 2 components, a callback definition(@app.callback and a callback function after tge callback definition.)

The input and the output of the callback function must match the definition of @app.callback

The output is followed by one or more inputs.

In [10]:
# callback definition for the scatter plot
@app.callback(
  Output('scatter-plot', 'figure'),
  Output('output_text_1', 'children'), #debug
  Input('year-slider', 'value'),
  Input('xaxis-column', 'value'),
  Input('yaxis-column', 'value'),
  Input('xaxis-type', 'value'),
  Input('yaxis-type', 'value'),
)



# first callback function
def update_graph(selected_year, xaxis_column_name, yaxis_column_name, xaxis_type, yaxis_type):
  # print all input params to keep track of the clicks and coordinates
  debug_params =''
    #'Input: {0}, {1}, {2}, {3}, {4}'.format(selected_year, xaxis_column_name, yaxis_column_name, xaxis_type, yaxis_type)

  # filter data frame by year
  filtered_df = df_country[df_country.year == selected_year]

  fig_scatter_plot = px.scatter(
    data_frame = filtered_df,
    x=str(xaxis_column_name),
    y=str(yaxis_column_name),
    hover_name="country",
    color="continent",
    size_max=55,
    
    custom_data = ["iso_alpha"],
    title= "{0}  vs {1} of Countries".format(xaxis_column_name, yaxis_column_name)
  )

  fig_scatter_plot.update_layout(transition_duration=500)

  fig_scatter_plot.update_xaxes(
    title=xaxis_column_name,
    type='linear' if xaxis_type == 'Linear' else 'log'
  )

  fig_scatter_plot.update_yaxes(
    title=yaxis_column_name,
    type='linear' if yaxis_type == 'Linear' else 'log'
  )

  # return
  return fig_scatter_plot, debug_params
# end update_








# second callback for the bar plot
@app.callback(
  Output('bar-chart', 'figure'),
  Output('output_text_2', 'children'), #debug
  Input('scatter-plot', 'clickData'), 
  Input('xaxis-column', 'value'),
  Input('xaxis-type', 'value')
)
# second callback definition
def update_bar_graph(clickData, xaxis_column_name, axis_type):
  if not clickData:
    return no_update

  debug_params = '' #using the debug params to keep track of the click data points
    #'Input: {0}, {1}, {2}'.format(clickData['points'], xaxis_column_name, axis_type)

  #print(str(clickData['points'][0]['customdata'][0]))
  country_code = str(clickData['points'][0]['customdata'][0])

  filtered_df = df_country[df_country['iso_alpha'] == country_code]

  fig_bar_plot = px.bar(
    data_frame = filtered_df,
    x="year",
    y=str(xaxis_column_name),
    title= "{0} of {1} ".format(xaxis_column_name, get_country_name(country_code))
  )

  fig_bar_plot.update_yaxes(
    title=xaxis_column_name,
    type='linear' if axis_type == 'Linear' else 'log'
  )

  # return
  return fig_bar_plot, debug_params


# end

## How to use the Dashboard?

##### 1. Clicking on any point of the scatterplot loads the barplot on the right side.

##### 2. The drop downs contain the different indicator values which loads the scatter plot. Selecting them is useful to populate the graph with appropriate information.

#####  3.The slider at the bottom of the scatter plot shows the indicator value for the specified year. 

##### 4. Double Clicking on the legends(continent) will highlight the scatterplot of  the specific continent. 

##### 5. The specific values for the data point is shown upon hovering on the plots.( Both for the scatter plot as wellas the Bar Plot)

##### 6. The graph can be viewed lineraly or logarithmically using the radio buttons at the bottom of the drop down.
Running the dash app on port 8050(my local machine)

In [11]:
app.run_server(mode='inline')
# if __name__ == '__main__':
#   app.run_server(debug=True)

## Contextual Visualizations
The below visualizations are plotted using the country_codes, latitudes and longitudes available in the countries_codes_and_coordinates.csv file along with the indicators from the gapminder file. 



In [12]:
#cleaning the dataframe columns to extract the latitude and longitude in appropriate format
df_country_code.rename(index=str,columns={"Latitude (average)":"lat","Longitude (average)":"lon"},inplace=True)
df_country_code.sample(10)

df_country_code["lat"] = df_country_code["lat"].str.replace('"', '')
df_country_code["lon"] = df_country_code["lon"].str.replace('"', '')

df_country_code["lat"] = df_country_code["lat"].astype(float)
df_country_code["lon"] = df_country_code["lon"].astype(float)


In [13]:
df_country_code.dtypes

Country          object
Alpha-2 code     object
Alpha-3 code     object
Numeric code     object
lat             float64
lon             float64
dtype: object

The Below plot shows the latitude, longitude and the countries that are found in the dataset. It is useful to understand how much data we have collected to draw conclusions.

In [14]:
import geopandas as gpd
#px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_geo(df_country_code,
                    lat=df_country_code.lat,
                    lon=df_country_code.lon,
                    hover_name="Country"
                    )
#fig.show()
#fig.write_html("file1.html")


#### Click this  link to see the <a href="https://thashmadech.github.io/thashmadech.github.io-img1/" target="_blank">co-ordinates of the data from where the data is gathered</a>



The below image shows the Population changes across different years across different countries and continents. Since each continent is represented by a color we can see that which continent's population is the highest.

##### The play button at the bottom of the plot is used for animating the world map to show the change in population over the years.

In [15]:
fig = px.scatter_geo(df_country, locations="iso_alpha",
                     color="continent", # which column to use to set the color of markers
                     hover_name="country", # column added to hover information
                     size="pop", # size of markers
                     projection="natural earth",
                     animation_frame="year"
                    )
#fig.show()
#fig.write_html("file2.html")


#### Click this  link to see the <a href="https://thashmadech.github.io/thashmadech.github.io-img2/" target="_blank">change in population over the years.</a>

The next plot is a gradient map displaying the life expectancy over the years.

##### The play button at the bottom of the plot is used for animating the world map to show the change in life expectancy over the years.

In [16]:

fig = px.choropleth(df_country, locations="iso_alpha", color="lifeExp", hover_name="country", animation_frame="year", range_color=[20,80])
#fig.show()
#fig.write_html("file3.html")


#### Click this  link to see the <a href="https://thashmadech.github.io/thashmadech.github.io-img3/" target="_blank">change in life expectancy over the years.</a>

From this assignment I learned a new python plotly library called dash which can be used to build apps with dashboards that can be run on browsers and IDE's.
While building the dashboard I encountered a lot of challenges and obstacles as this is my first time using this library. Watching tutorial videos and going through sample sources available online helped me overcome these challenges as well as learn new ways of creating visualizations.  

##### Since I was not able to render the plotly express contextual visualizations in github pages I have provided links to them above.It takes you to the visualizations I have created and uploaded separately in my github repo.Clicking on the links will display the visualizations in a new tab.

 #####  References:
    
 https://medium.com/analytics-vidhya/express-way-to-do-visualization-in-python-b9bc01143553
 
 
 https://www.kaggle.com/code/jhossain/explore-the-gapminder-dataset-with-plotly-express/notebook
 
 
 https://plotly.com/python/plotly-express/
 https://www.gapminder.org/tools/#$model$markers$bubble$encoding$selected$data$filter$markers@=ind&=tur&=egy&=uga&=kornga;;;;;;;;&chart-type=map&url=v1
 
 
 https://plotly.com/python/scatter-plots-on-maps/    
 
https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Deploy_App_to_Web/Kaggle-GGL-Collab/Dash-on-GGL-Colab.ipynb

https://www.justintodata.com/python-interactive-dashboard-with-plotly-dash-tutorial/

https://dash.plotly.com/dash-core-components


In [18]:
#converting the jupyter notebook into html
#! jupyter nbconvert --to html nellira-ragunanda-thashma-final_project_formatted.ipynb

[NbConvertApp] Converting notebook nellira-ragunanda-thashma-final_project_formatted.ipynb to html
[NbConvertApp] Writing 630920 bytes to nellira-ragunanda-thashma-final_project_formatted.html


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
12,Australia,"""AU""",AUS,"""36""",-27.0,133.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
12,Australia,"""AU""",AUS,"""36""",-27.0,133.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
236,Vietnam,"""VN""",VNM,"""704""",16.0,106.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
236,Vietnam,"""VN""",VNM,"""704""",16.0,106.0


...

,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
226,Uganda,"""UG""",UGA,"""800""",1.0,32.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
145,Montenegro,"""ME""",MNE,"""499""",42.0,19.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
209,Sweden,"""SE""",SWE,"""752""",62.0,15.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
209,Sweden,"""SE""",SWE,"""752""",62.0,15.0


,Country,Alpha-2 code,Alpha-3 code,Numeric code,lat,lon
220,Trinidad and Tobago,"""TT""",TTO,"""780""",11.0,-61.0
